In [ ]:
%pip install pandas

In [ ]:
import pandas as pd
from collections import defaultdict
from typing import Iterator, Tuple, Dict, List, Any

pd.set_option('display.max_columns', None)

In [3]:
# --- MAP ЭТАП ---
def map_user_activity(row: Dict[str, Any]) -> Iterator[Tuple[str, float]]:
    """
    Map функция: извлекает ID пользователя и поставленную им оценку.
    """
    user_id = row.get('User_id', 'Unknown')
    score = row.get('review/score', 0)
    
    # Пропускаем, если ID нет или оценка некорректна
    if not isinstance(user_id, str) or user_id == 'Unknown':
        return
    
    try:
        score_float = float(score)
        yield (user_id, score_float)
    except (ValueError, TypeError):
        return

# --- SHUFFLE ЭТАП ---
def shuffle_user_activity(mapped_data: Iterator[Tuple[str, float]]) -> Dict[str, List[float]]:
    """
    Shuffle функция: группирует все оценки по каждому пользователю.
    """
    groups = defaultdict(list)
    for user, score in mapped_data:
        groups[user].append(score)
    return groups

# --- REDUCE ЭТАП ---
def reduce_user_activity(grouped_data: Dict[str, List[float]]) -> List[Tuple[str, int, float]]:
    """
    Reduce функция: считает количество рецензий и среднюю оценку для каждого пользователя.
    Возвращает: (User_id, Количество рецензий, Средняя оценка)
    """
    results = []
    for user, scores in grouped_data.items():
        count = len(scores)
        avg_score = sum(scores) / count if count > 0 else 0
        results.append((user, count, avg_score))
    return results

In [4]:
# --- MAP ЭТАП ---
def map_publisher_rating(row: Dict[str, Any]) -> Iterator[Tuple[str, float]]:
    """
    Map функция: извлекает Издателя и оценку.
    """
    publisher = row.get('publisher', '')
    score = row.get('review/score', 0)
    
    # Очистка данных: убираем пустые значения и мусор
    if not isinstance(publisher, str) or not publisher.strip():
        return
        
    try:
        score_float = float(score)
        # Немного приводим название к единому виду
        clean_publisher = publisher.strip().strip("['\"]")
        yield (clean_publisher, score_float)
    except (ValueError, TypeError):
        return

# --- SHUFFLE ЭТАП ---
def shuffle_publisher_rating(mapped_data: Iterator[Tuple[str, float]]) -> Dict[str, List[float]]:
    """
    Shuffle функция: группирует оценки по издателям.
    """
    groups = defaultdict(list)
    for pub, score in mapped_data:
        groups[pub].append(score)
    return groups

# --- REDUCE ЭТАП ---
def reduce_publisher_rating(grouped_data: Dict[str, List[float]], min_reviews: int = 5) -> List[Tuple[str, float, int]]:
    """
    Reduce функция: считает средний рейтинг издательства.
    Отсеиваем издательства, у которых мало отзывов (меньше min_reviews).
    """
    results = []
    for pub, scores in grouped_data.items():
        if len(scores) >= min_reviews: # Фильтрация для достоверности
            avg_score = sum(scores) / len(scores)
            results.append((pub, avg_score, len(scores)))
    return results

In [ ]:
def load_dataset(limit: int = None) -> pd.DataFrame:
    try:
        print("Загрузка данных...")
        # Читаем основные файлы
        df_reviews = pd.read_csv('Books_rating.csv')
        df_meta = pd.read_csv('Books_data.csv')
        
        # Для анализа издательств нам нужен Merge двух таблиц по названию книги (Title)
        # Используем left join, чтобы не потерять рецензии
        merged_df = pd.merge(df_reviews, df_meta[['Title', 'publisher']], on='Title', how='left')
        
        if limit:
            return merged_df.head(limit)
        return merged_df
        
    except FileNotFoundError:
        print("Файлы не найдены! Убедитесь, что dataset скачан с Kaggle.")
        return pd.DataFrame()

# Загружаем часть данных для демонстрации
df = load_dataset(limit=50000)
print(f"Данные загружены. Размер: {df.shape}")

Загрузка данных...
Данные загружены. Размер: (50000, 11)


In [ ]:
print("\n=== ЗАДАЧА 1: ТОП АКТИВНЫХ ПОЛЬЗОВАТЕЛЕЙ ===")
# 1. Map
mapped_users = []
for _, row in df.iterrows():
    mapped_users.extend(map_user_activity(row))

# 2. Shuffle
shuffled_users = shuffle_user_activity(iter(mapped_users))

# 3. Reduce
reduced_users = reduce_user_activity(shuffled_users)

# 4. Ранжирование (Ranking)
top_users = sorted(reduced_users, key=lambda x: x[1], reverse=True)[:10]

print(f"{'USER ID':<20} | {'COUNT':<10} | {'AVG SCORE':<10}")
print("-" * 45)
for user, count, avg in top_users:
    print(f"{user[:18]:<20} | {count:<10} | {avg:.2f}")


print("\n\n=== ЗАДАЧА 2: ТОП КАЧЕСТВЕННЫХ ИЗДАТЕЛЬСТВ ===")
# 1. Map
mapped_pubs = []
for _, row in df.iterrows():
    mapped_pubs.extend(map_publisher_rating(row))

# 2. Shuffle
shuffled_pubs = shuffle_publisher_rating(iter(mapped_pubs))

# 3. Reduce
reduced_pubs = reduce_publisher_rating(shuffled_pubs, min_reviews=10)

# 4. Ранжирование (Ranking)
top_publishers = sorted(reduced_pubs, key=lambda x: x[1], reverse=True)[:10]

print(f"{'PUBLISHER':<40} | {'RATING':<10} | {'REVIEWS':<10}")
print("-" * 65)
for pub, rating, count in top_publishers:
    print(f"{pub[:38]:<40} | {rating:.2f}       | {count:<10}")


=== ЗАДАЧА 1: ТОП АКТИВНЫХ ПОЛЬЗОВАТЕЛЕЙ ===
USER ID              | COUNT      | AVG SCORE 
---------------------------------------------
A14OJS0VWMOSWO       | 99         | 5.00
AFVQZQ8PW0L          | 46         | 4.67
A1D2C0WDCSHUWZ       | 43         | 4.60
A1L43KWWR05PCS       | 41         | 4.41
AHD101501WCN1        | 39         | 4.59
A1X8VZWTOG8IS6       | 35         | 3.80
A20EEWWSFMZ1PN       | 30         | 4.93
A1EKTLUL24HDG8       | 27         | 4.67
A3MV1KKHX51FYT       | 23         | 4.30
A1N1YEMTI9DJ86       | 23         | 4.52


=== ЗАДАЧА 2: ТОП КАЧЕСТВЕННЫХ ИЗДАТЕЛЬСТВ ===
PUBLISHER                                | RATING     | REVIEWS   
-----------------------------------------------------------------
Longtail Publishing                      | 5.00       | 22        
University of Central Florida            | 5.00       | 10        
Thomas Nelson                            | 5.00       | 11        
Dial Press Trade Paperback               | 5.00       | 15        
U